In [1]:
# -*- coding: utf-8 -*-
# Requerimientos:
!pip install pytrends pandas
from pytrends.request import TrendReq
import pandas as pd
import time
import os
import random
from datetime import datetime
from google.colab import drive

# Crear instancia global
pytrends = TrendReq(hl='es-ES', tz=0)

# Carpeta donde se guardarán los archivos
save_dir = "/content/drive/MyDrive/Google Trends"
os.makedirs(save_dir, exist_ok=True)

# Montar Google Drive para persistencia
drive.mount('/content/drive')

# === CONFIGURACIÓN ===
TIMEFRAME = 'now 7-d'  # Última semana
GEO = 'CO'

CANDIDATES = {
    "María José Pizarro": "/g/11gy75qcnw",
    "Gustavo Bolívar": "/m/02vvjjd",
    "Daniel Quintero": "/g/11h75kc2ys",
    "Luis Gilberto Murillo": "/m/0417wps",
    "Mauricio Lizcano": "/g/11b5q7z1ch",
    "Roy Barreras": "/g/12215q6r",
    "Claudia López": "/g/11h1t55sk",
    "Juan Manuel Galán": "/m/0w5vv7s",
    "Sergio Fajardo": "/m/02vyn0k",
    "Juan Daniel Oviedo": "/g/11h8vbrl8r",
    "Alejandro Gaviria": "/m/0gvqzbs",
    "Germán Vargas Lleras": "/m/03d8kw1",
    "David Luna": "/g/120wkspn",
    "Vicky Dávila": "/g/121p6qfk",
    "María Fernanda Cabal": "/g/1q6mq6sx9",
    "Mauricio Cárdenas": "/m/05p699p",
    "Jota Pe Hernández": "/g/11q4c11tj_",
    "Santiago Botero": "/g/11mct_q0pt",
    "Felipe Córdoba": "/g/11gm7pl53m",
    "Francisco Barbosa": "/g/113s4tpzs",
    "Gustavo Petro": "/m/03c3tmt",
    "Abelardo de la Espriella": "/g/11bwfmp95b",
    "Héctor Olimpo": "/g/11px8xgf6m",
    "Juan Guillermo Zuluaga": "/g/11fz9ysx4d",
    "Daniel Palacios": "/g/11llgb68wy",
    "Paloma Valencia": "/g/11b5qbb4pl",
    "Aníbal Gaviria": "/m/0j_6b8n",
    "Juan Carlos Cárdenas": "/g/11hf03jqyp",
    "Juan Carlos Saldarriaga": "/g/11fp8wb95q",
    "Álvaro Uribe": "/m/01kb1b"
}

# === FUNCIONES AUXILIARES ===
def append_to_files(new_df, base_filename):
    csv_path = os.path.join(save_dir, base_filename + ".csv")
    parquet_path = os.path.join(save_dir, base_filename + ".parquet")

    # Guardar CSV
    if os.path.exists(csv_path):
        old_df_csv = pd.read_csv(csv_path)
        combined_df_csv = pd.concat([old_df_csv, new_df], ignore_index=True)
    else:
        combined_df_csv = new_df
    combined_df_csv.to_csv(csv_path, index=False)

    # Guardar Parquet
    if os.path.exists(parquet_path):
        old_df_parquet = pd.read_parquet(parquet_path)
        combined_df_parquet = pd.concat([old_df_parquet, new_df], ignore_index=True)
    else:
        combined_df_parquet = new_df
    combined_df_parquet.to_parquet(parquet_path, index=False)

    print(f"✅ Guardado en {csv_path} y {parquet_path} ({len(new_df)} filas nuevas)")

def safe_request(func, *args, **kwargs):
    for attempt in range(3):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if "429" in str(e):
                wait_time = random.randint(180, 300)
                print(f"⚠️ Bloqueado por Google (429). Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"❌ Error: {e}")
                return pd.DataFrame()
    print("❌ Falló después de varios intentos.")
    return pd.DataFrame()

def get_interest_over_time(name, topic_id):
    pytrends.build_payload([topic_id], timeframe=TIMEFRAME, geo=GEO)
    data = pytrends.interest_over_time().drop(columns=['isPartial'], errors='ignore')
    data.reset_index(inplace=True)
    data.rename(columns={topic_id: 'Interest'}, inplace=True)
    data['Candidate'] = name
    data['Collected_Date'] = datetime.today().strftime('%Y-%m-%d')
    return data

def get_interest_by_region(name, topic_id):
    pytrends.build_payload([topic_id], timeframe=TIMEFRAME, geo=GEO)
    data = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=True).reset_index()
    data.rename(columns={topic_id: 'Interest'}, inplace=True)
    data['Candidate'] = name
    data['Collected_Date'] = datetime.today().strftime('%Y-%m-%d')
    return data

def get_top_queries(name, topic_id):
    pytrends.build_payload([topic_id], timeframe=TIMEFRAME, geo=GEO)
    related_queries_raw = pytrends.related_queries()
    if isinstance(related_queries_raw, dict) and topic_id in related_queries_raw:
        candidate_data = related_queries_raw[topic_id]
        if isinstance(candidate_data, dict) and isinstance(candidate_data.get('top'), pd.DataFrame):
            top_df = candidate_data['top'][['query', 'value']].rename(columns={'query': 'Query', 'value': 'Interest'})
            top_df['Candidate'] = name
            top_df['Collected_Date'] = datetime.today().strftime('%Y-%m-%d')
            return top_df
    return pd.DataFrame()

# === PROCESO PRINCIPAL ===
def collect_google_trends():
    print(f"\n=== Recolección iniciada: {datetime.now()} ===")
    for name, topic_id in CANDIDATES.items():
        print(f"\n🔍 Procesando: {name}")

        time_df = safe_request(get_interest_over_time, name, topic_id)
        if not time_df.empty:
            append_to_files(time_df, "interest_over_time")

        region_df = safe_request(get_interest_by_region, name, topic_id)
        if not region_df.empty:
            append_to_files(region_df, "interest_by_region")

        top_df = safe_request(get_top_queries, name, topic_id)
        if not top_df.empty:
            append_to_files(top_df, "top_queries")

        pause_time = random.randint(40, 70)
        print(f"⏳ Esperando {pause_time} segundos antes del siguiente candidato...")
        time.sleep(pause_time)

    print(f"\n✅ Recolección finalizada: {datetime.now()}")

# === EJECUTAR ===
collect_google_trends()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== Recolección iniciada: 2025-08-14 20:54:15.760815 ===

🔍 Procesando: María José Pizarro


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


✅ Guardado en /content/drive/MyDrive/Google Trends/interest_over_time.csv y /content/drive/MyDrive/Google Trends/interest_over_time.parquet (169 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/top_queries.csv y /content/drive/MyDrive/Google Trends/top_queries.parquet (25 filas nuevas)
⏳ Esperando 40 segundos antes del siguiente candidato...

🔍 Procesando: Gustavo Bolívar


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


✅ Guardado en /content/drive/MyDrive/Google Trends/interest_over_time.csv y /content/drive/MyDrive/Google Trends/interest_over_time.parquet (169 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/top_queries.csv y /content/drive/MyDrive/Google Trends/top_queries.parquet (25 filas nuevas)
⏳ Esperando 69 segundos antes del siguiente candidato...

🔍 Procesando: Daniel Quintero


...

✅ Guardado en /content/drive/MyDrive/Google Trends/interest_over_time.csv y /content/drive/MyDrive/Google Trends/interest_over_time.parquet (169 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
⏳ Esperando 54 segundos antes del siguiente candidato...

🔍 Procesando: Juan Carlos Cárdenas
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
⏳ Esperando 60 segundos antes del siguiente candidato...

🔍 Procesando: Juan Carlos Saldarriaga


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


✅ Guardado en /content/drive/MyDrive/Google Trends/interest_over_time.csv y /content/drive/MyDrive/Google Trends/interest_over_time.parquet (169 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
⏳ Esperando 50 segundos antes del siguiente candidato...

🔍 Procesando: Álvaro Uribe


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


✅ Guardado en /content/drive/MyDrive/Google Trends/interest_over_time.csv y /content/drive/MyDrive/Google Trends/interest_over_time.parquet (169 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/interest_by_region.csv y /content/drive/MyDrive/Google Trends/interest_by_region.parquet (33 filas nuevas)
✅ Guardado en /content/drive/MyDrive/Google Trends/top_queries.csv y /content/drive/MyDrive/Google Trends/top_queries.parquet (25 filas nuevas)
⏳ Esperando 56 segundos antes del siguiente candidato...

✅ Recolección finalizada: 2025-08-14 21:22:47.994766
